# Revolve Tatbletennis ELO ladder
# Author: Harald Lønsethagen
# Started: 28.08.2017

## Importing Libraries

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime

## Defining usefull functions

In [15]:
def expected(A, B):
    """
    Calculate expected score of A in a match against B
    :param A: Elo rating for player A
    :param B: Elo rating for player B
    """
    return 1 / (1 + 10 ** ((B - A) / 400))


def elo(old, exp, score, k=32):
    """
    Calculate the new Elo rating for a player
    :param old: The previous Elo rating
    :param exp: The expected score for this match
    :param score: The actual score for this match
    :param k: The k-factor for Elo (default: 32)
    """
    return old + k * (score - exp)

## Importing all played matches

In [16]:
matches_df = pd.read_csv("matches.csv")
print("Matches: " + str(matches_df.shape[0]))
matches_df

Matches: 288


,Player 1,Player 2,Winner,timestamp
0,Paul Frivold,Robert Tamang,Robert Tamang,2017-09-08
1,Paul Frivold,Bo Woelfert,Bo Woelfert,2017-09-08
2,Mons,Ole Andreas,Mons,2017-09-09
3,Mons,Eirik B.,Mons,2017-09-09
4,Mons,Eirik B.,Mons,2017-09-09
5,Paul Frivold,Dino,Paul Frivold,2017-09-09
6,Paul Frivold,Mons,Mons,2017-09-09
7,Bo Woelfert,Dino,Bo Woelfert,2017-09-09
8,Jakob,Mons,Mons,2017-09-10
9,Mons,Yohann,Mons,2017-09-10


## Importing all players 

In [17]:
players_df = pd.read_csv("players.csv")
print("Players: " + str(players_df.shape[0]))
players_df

Players: 53


,first_name,last_name,short_name,joined_date,id
0,Amund,NaN,rt,17-09-08,1
1,Anders,NaN,mbe,17-09-08,2
2,Andreas B.,NaN,NaN,17-09-07,3
3,Andreas H.,NaN,NaN,17-09-06,4
4,Bo Woelfert,NaN,NaN,17-09-06,5
5,Didrik,NaN,NaN,17-09-04,6
6,Dino,NaN,NaN,17-09-03,7
7,Eirik B.,NaN,NaN,17-09-02,8
8,Even,NaN,NaN,17-09-02,9
9,Fabian S. Rønningen,NaN,NaN,17-09-01,10


## ELO-ladder Dataframe containing the ELO-score to each player

In [18]:
elo_df = players_df
elo_df['elo'] = 1000
elo_df['matches_played'] = 0
elo_df['wins'] = 0
elo_df['losses'] = 0
elo_df['win_rate'] = 0.1321321321
print("Size of ELO-df: " + str(elo_df.shape[0]))
elo_df.head()

Size of ELO-df: 53


,first_name,last_name,short_name,joined_date,id,elo,matches_played,wins,losses,win_rate
0,Amund,NaN,rt,17-09-08,1,1000,0,0,0,0.132132
1,Anders,NaN,mbe,17-09-08,2,1000,0,0,0,0.132132
2,Andreas B.,NaN,NaN,17-09-07,3,1000,0,0,0,0.132132
3,Andreas H.,NaN,NaN,17-09-06,4,1000,0,0,0,0.132132
4,Bo Woelfert,NaN,NaN,17-09-06,5,1000,0,0,0,0.132132


In [19]:
# New df for plotting
plott_df = players_df.set_index("first_name").T
plott_df["time"] = 0
plott_df = plott_df.reset_index()
append_df = pd.DataFrame([["2017-09-08"]], columns=["time"])
plott_df = plott_df.append(append_df)
plott_df = plott_df.iloc[-1:]
plott_df

,Aida,Aksel,Amund,Anders,Andreas B.,Andreas H.,Bo Woelfert,Cornelia,Didrik,Dino,...,Sondre N.A.,Thomas F,Thomas H,Thomas O,Truls,Yohann,index,time,Øystein,Øyvind S.
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-08,NaN,NaN


# Function for calculating ELO

In [20]:
matches_df.iloc(1)[0]

Player 1      Paul Frivold
Player 2     Robert Tamang
Winner       Robert Tamang
timestamp       2017-09-08
Name: 0, dtype: object

In [21]:
biggest_upset_win_increase = 0
biggest_upset_loss_decrease = 0
biggest_upset_player_win = ""
biggest_upset_player_loss = ""
biggest_upset_player_date = ""
last_date = matches_df.iloc[0]['timestamp']

length = matches_df.shape[0]
print("Number of matches: " + str(length) + "\n")
for i in range(length):
    match = matches_df.iloc[i]
    #print(str(i) +": " + str(match['Player 1']) + " - " + str(match['Player 2']))
    player_1 = match["Player 1"]
    player_2 = match["Player 2"]
    elo_rating_a = elo_df[elo_df['first_name'] == player_1]['elo']
    elo_rating_b = elo_df[elo_df['first_name'] == player_2]['elo']
    
    if(elo_rating_a.shape[0] == 0):
        print("========================================================================")
        print("Player named '" + str(player_1) + "' is NOT FOUND in the player database. ")
        print("========================================================================")
        break
    if(elo_rating_b.shape[0] == 0):
        print("========================================================================")
        print("Player named '" + str(player_2) + "' is NOT FOUND in the player database. ")
        print("========================================================================")
        break
    elo_rating_a = int(elo_rating_a.values[0])
    elo_rating_b = int(elo_rating_b.values[0])
    
    
    if(match['Winner'] == player_1): 
        winner_a = 1
        winner_b = 0
    else:
        winner_a = 0
        winner_b = 1
    expected_a  = expected(elo_rating_a, elo_rating_b)
    expected_b  = expected(elo_rating_b, elo_rating_a)
    
    new_elo_a = elo(elo_rating_a, expected_a, winner_a, k = 50)
    new_elo_b = elo(elo_rating_b, expected_b, winner_b, k = 50)
    
    if new_elo_a - elo_rating_a > biggest_upset_win_increase:
        # New biggest upset detected
        biggest_upset_win_increase = round(new_elo_a - elo_rating_a,2)
        biggest_upset_loss_decrease = round(elo_rating_b - new_elo_b,2)
        biggest_upset_player_win = player_1
        biggest_upset_player_loss = player_2
        biggest_upset_player_date = match["timestamp"]
    elif new_elo_b - elo_rating_b > biggest_upset_win_increase:
        # New biggest upset detected
        biggest_upset_win_increase = round(new_elo_b - elo_rating_b, 2)
        biggest_upset_loss_decrease = round(elo_rating_a - new_elo_a,2)
        biggest_upset_player_win = player_2
        biggest_upset_player_loss = player_1
        biggest_upset_player_date = match["timestamp"]
        
    
    #print("New elo a: " + str(new_elo_a))
    #print("New elo b: " + str(new_elo_b))
    #print("Player 1: '" + player_1 + "'. Old elo: " + str(elo_rating_a) + "'. New elo: " + str(new_elo_a) + ". exp: " + str(expected_a) + " " )
    #print("Player 1: '" + player_2 + "'. Old elo: " + str(elo_rating_b) + "'. New elo: " + str(new_elo_b) + ". exp: " + str(expected_b) + " " )
    elo_df = elo_df.set_index('first_name')
    elo_df = elo_df.set_value(player_1, 'elo', new_elo_a + 1)
    elo_df = elo_df.set_value(player_2, 'elo', new_elo_b + 1)
    elo_df = elo_df.reset_index()
    
    # plotting df updates
    current_date = match['timestamp']
    if last_date == current_date:
        plott_df = plott_df.set_index("time")
        plott_df.set_value(match['timestamp'], player_1, new_elo_a)
        plott_df.set_value(match['timestamp'], player_2, new_elo_b)
        plott_df = plott_df.reset_index()
    else:
        append_df = pd.DataFrame([[match['timestamp']]], columns=["time"])
        plott_df = plott_df.append(append_df)
        plott_df = plott_df.set_index("time")
        plott_df.set_value(match['timestamp'], player_1, new_elo_a)
        plott_df.set_value(match['timestamp'], player_2, new_elo_b)
        plott_df = plott_df.reset_index()
        last_date = current_date
        
    
plott_df = plott_df.fillna(method="ffill")
elo_df.head()

Number of matches: 288



,first_name,last_name,short_name,joined_date,id,elo,matches_played,wins,losses,win_rate
0,Amund,NaN,rt,17-09-08,1,980,0,0,0,0.132132
1,Anders,NaN,mbe,17-09-08,2,939,0,0,0,0.132132
2,Andreas B.,NaN,NaN,17-09-07,3,1186,0,0,0,0.132132
3,Andreas H.,NaN,NaN,17-09-06,4,1026,0,0,0,0.132132
4,Bo Woelfert,NaN,NaN,17-09-06,5,1095,0,0,0,0.132132


In [22]:
dates = []
for date in plott_df["time"]:
    dates.append(datetime.strptime(date, '%Y-%m-%d'))
print(plott_df.iloc[0]["time"])
dates

2017-09-08


[datetime.datetime(2017, 9, 8, 0, 0),
 datetime.datetime(2017, 9, 9, 0, 0),
 datetime.datetime(2017, 9, 10, 0, 0),
 datetime.datetime(2017, 9, 11, 0, 0),
 datetime.datetime(2017, 9, 12, 0, 0),
 datetime.datetime(2017, 9, 13, 0, 0),
 datetime.datetime(2017, 9, 14, 0, 0),
 datetime.datetime(2017, 9, 17, 0, 0),
 datetime.datetime(2017, 9, 18, 0, 0),
 datetime.datetime(2017, 9, 19, 0, 0),
 datetime.datetime(2017, 9, 20, 0, 0),
 datetime.datetime(2017, 9, 21, 0, 0),
 datetime.datetime(2017, 9, 22, 0, 0),
 datetime.datetime(2017, 9, 23, 0, 0),
 datetime.datetime(2017, 9, 24, 0, 0),
 datetime.datetime(2017, 9, 25, 0, 0),
 datetime.datetime(2017, 9, 26, 0, 0),
 datetime.datetime(2017, 9, 27, 0, 0),
 datetime.datetime(2017, 9, 28, 0, 0),
 datetime.datetime(2017, 9, 29, 0, 0),
 datetime.datetime(2017, 10, 1, 0, 0),
 datetime.datetime(2017, 10, 2, 0, 0),
 datetime.datetime(2017, 10, 3, 0, 0),
 datetime.datetime(2017, 10, 4, 0, 0),
 datetime.datetime(2017, 10, 5, 0, 0),
 datetime.datetime(2017, 10

In [23]:
dates = []
for date in plott_df["time"]:
    dates.append(datetime.strptime(date, '%Y-%m-%d'))
dates
def plotPlayers(players):
    
    plt.close('all')
    height = 19
    width = 38
    fig = plt.figure(figsize=(width,height))
    #figsize=(38,19) for 2189 x 1229 resoltion. Which is perfect for fullscreen 
    

    for player in players:
        plt.plot( dates, plott_df[player] , label = player)
    plt.legend(loc='best')

    # ==== Set figure title and labels ====
    fig.suptitle("Temporal ELO plots", fontsize=35)
    plt.xlabel('Time', fontsize=25)
    plt.ylabel('ELO', fontsize=25)
    plt.grid()
    matplotlib.rcParams.update({'font.size': 17})

    # ==== Save file ====
    fig.savefig("plots/" +str(players.values[:]) + ".png", bbox_inches='tight')
    # ---- Clear 


In [135]:
print(plott_df.columns[1:].values[:])

['Aida' 'Aksel' 'Amund' 'Anders' 'Andreas B.' 'Andreas H.' 'Bo Woelfert'
 'Cornelia' 'Didrik' 'Dino' 'Edvard N' 'Eirik' 'Eirik B.' 'Emil' 'Even'
 'Fabian S. Rønningen' 'Fredrik C' 'Hans' 'Harald Lønsethagen' 'Henrik'
 'Håkon Aero' 'Håkon M.' 'Jakob' 'John' 'Kristoffer H' 'Lars Erik' 'Lasse'
 'Magnus Aero' 'Magnus G.L.' 'Magnus K.B.' 'Magnus Leinam' 'Marcus'
 'Martine' 'Mathias' 'Mats' 'Mia' 'Mons' 'Odin A.S.' 'Ole Andreas'
 'Ole Marius' 'Paul Frivold' 'Robert Tamang' 'Sisse' 'Sondre'
 'Sondre Midtskogen' 'Sondre N.A.' 'Thomas F' 'Thomas H' 'Thomas O' 'Truls'
 'Yohann' 'index' 'Øystein' 'Øyvind S.']


In [11]:
plott_df.columns[0:10]

Index(['Aida', 'Aksel', 'Amund', 'Anders', 'Andreas B.', 'Andreas H.',
       'Bo Woelfert', 'Cornelia', 'Didrik', 'Dino'],
      dtype='object')

In [139]:
plotPlayers(plott_df.columns[1:10])
plotPlayers(plott_df.columns[10:20])
plotPlayers(plott_df.columns[20:30])
plotPlayers(plott_df.columns[40:50])
plotPlayers(plott_df.columns[50:60])
plotPlayers(plott_df.columns[50:60])

In [163]:
plotPlayers(latex_df.iloc[:10]["name"])
plotPlayers(latex_df_2.iloc[-10:]["name"])


In [162]:
latex_df_2.iloc[-10:]["name"]

0     Eirik B.
1         Dino
2         Emil
3      Mathias
4    Odin A.S.
Name: name, dtype: object

In [153]:
elo_df

,first_name,last_name,short_name,joined_date,id,elo,matches_played,wins,losses,win_rate
0,Amund,NaN,rt,17-09-08,1,980,0,0,0,0.132132
1,Anders,NaN,mbe,17-09-08,2,939,0,0,0,0.132132
2,Andreas B.,NaN,NaN,17-09-07,3,1186,0,0,0,0.132132
3,Andreas H.,NaN,NaN,17-09-06,4,1026,0,0,0,0.132132
4,Bo Woelfert,NaN,NaN,17-09-06,5,1095,0,0,0,0.132132
5,Didrik,NaN,NaN,17-09-04,6,1020,0,0,0,0.132132
6,Dino,NaN,NaN,17-09-03,7,891,0,0,0,0.132132
7,Eirik B.,NaN,NaN,17-09-02,8,894,0,0,0,0.132132
8,Even,NaN,NaN,17-09-02,9,967,0,0,0,0.132132
9,Fabian S. Rønningen,NaN,NaN,17-09-01,10,1114,0,0,0,0.132132


In [12]:
elo_df.columns

Index(['first_name', ' last_name', ' short_name', ' joined_date', ' id', 'elo',
       'matches_played', 'wins', 'losses', 'win_rate'],
      dtype='object')

In [24]:
length = matches_df.shape[0]
print("Number of matches: " + str(length) + "\n")
for i in range(length):
    match = matches_df.iloc[i]
    #print(str(i) +": '" + str(match['Player 1']) + "' - '" + str(match['Player 2']) + "'")
    player_1 = match["Player 1"]
    player_2 = match["Player 2"]
    old_number_of_matches_played_player_1 = elo_df[elo_df['first_name'] == player_1]['matches_played']
    old_number_of_matches_played_player_2 = elo_df[elo_df['first_name'] == player_2]['matches_played']
    if(match['Winner'] == player_1): 
        old_number_of_wins_player_1 = elo_df[elo_df['first_name'] == player_1]['wins']
        old_number_of_losses_player_2 = elo_df[elo_df['first_name'] == player_2]['losses']
        elo_df = elo_df.set_index('first_name')
        elo_df = elo_df.set_value(player_1, 'wins', old_number_of_wins_player_1 + 1)
        elo_df = elo_df.set_value(player_2, 'losses', old_number_of_losses_player_2 + 1)
        elo_df = elo_df.reset_index()
    else:
        old_number_of_wins_player_2 = elo_df[elo_df['first_name'] == player_2]['wins']
        old_number_of_losses_player_1 = elo_df[elo_df['first_name'] == player_1]['losses']
        elo_df = elo_df.set_index('first_name')
        elo_df = elo_df.set_value(player_2, 'wins', old_number_of_wins_player_2 + 1)
        elo_df = elo_df.set_value(player_1, 'losses', old_number_of_losses_player_1 + 1)
        elo_df = elo_df.reset_index()
    
    elo_df = elo_df.set_index('first_name')
    elo_df = elo_df.set_value(player_1, 'matches_played', old_number_of_matches_played_player_1 + 1)
    elo_df = elo_df.set_value(player_2, 'matches_played', old_number_of_matches_played_player_2 + 1)
    elo_df = elo_df.reset_index()
players = elo_df['first_name']
for player in players:
    #print(player)
    wins = elo_df[elo_df['first_name'] == player]['wins']
    losses = elo_df[elo_df['first_name'] == player]['losses']
    matches_played = elo_df[elo_df['first_name'] == player]['matches_played']
    win_rate = round((wins / matches_played) * 100,2)
    print(win_rate)
    elo_df = elo_df.set_index('first_name')
    elo_df = elo_df.set_value(player, 'win_rate', win_rate)
    elo_df = elo_df.reset_index()
elo_df

Number of matches: 288

0    38.46
dtype: float64
1    57.89
dtype: float64
2    91.3
dtype: float64
3    100.0
dtype: float64
4    65.79
dtype: float64
5    44.12
dtype: float64
6    33.33
dtype: float64
7    0.0
dtype: float64
8    27.27
dtype: float64
9    90.0
dtype: float64
10    59.09
dtype: float64
11    25.0
dtype: float64
12    0.0
dtype: float64
13    42.59
dtype: float64
14    0.0
dtype: float64
15    100.0
dtype: float64
16    0.0
dtype: float64
17    50.0
dtype: float64
18    6.67
dtype: float64
19    87.5
dtype: float64
20    14.81
dtype: float64
21    68.0
dtype: float64
22    33.33
dtype: float64
23    0.0
dtype: float64
24    85.37
dtype: float64
25    0.0
dtype: float64
26    53.33
dtype: float64
27    0.0
dtype: float64
28    66.67
dtype: float64
29    66.67
dtype: float64
30    43.75
dtype: float64
31    57.14
dtype: float64
32    33.33
dtype: float64
33    50.0
dtype: float64
34    33.33
dtype: float64
35    90.91
dtype: float64
36    0.0
dtype: float64
37    80.0


,first_name,last_name,short_name,joined_date,id,elo,matches_played,wins,losses,win_rate
0,Amund,NaN,rt,17-09-08,1,980,13,5,8,38.46
1,Anders,NaN,mbe,17-09-08,2,939,19,11,8,57.89
2,Andreas B.,NaN,NaN,17-09-07,3,1186,23,21,2,91.30
3,Andreas H.,NaN,NaN,17-09-06,4,1026,1,1,0,100.00
4,Bo Woelfert,NaN,NaN,17-09-06,5,1095,38,25,13,65.79
5,Didrik,NaN,NaN,17-09-04,6,1020,34,15,19,44.12
6,Dino,NaN,NaN,17-09-03,7,891,18,6,12,33.33
7,Eirik B.,NaN,NaN,17-09-02,8,894,5,0,5,0.00
8,Even,NaN,NaN,17-09-02,9,967,11,3,8,27.27
9,Fabian S. Rønningen,NaN,NaN,17-09-01,10,1114,10,9,1,90.00


In [155]:
elo_df

,first_name,last_name,short_name,joined_date,id,elo,matches_played,wins,losses,win_rate
0,Amund,NaN,rt,17-09-08,1,980,13,5,8,38.46
1,Anders,NaN,mbe,17-09-08,2,939,19,11,8,57.89
2,Andreas B.,NaN,NaN,17-09-07,3,1186,23,21,2,91.30
3,Andreas H.,NaN,NaN,17-09-06,4,1026,1,1,0,100.00
4,Bo Woelfert,NaN,NaN,17-09-06,5,1095,38,25,13,65.79
5,Didrik,NaN,NaN,17-09-04,6,1020,34,15,19,44.12
6,Dino,NaN,NaN,17-09-03,7,891,18,6,12,33.33
7,Eirik B.,NaN,NaN,17-09-02,8,894,5,0,5,0.00
8,Even,NaN,NaN,17-09-02,9,967,11,3,8,27.27
9,Fabian S. Rønningen,NaN,NaN,17-09-01,10,1114,10,9,1,90.00


In [134]:
elo_df.columns = ['first_name', 'last_name', 'short_name', 'joined_date', 'id',
       'elo']
elo_df.columns

ValueError: Length mismatch: Expected axis has 10 elements, new values have 6 elements

In [156]:
temp_df = elo_df.sort_values('elo', ascending=False)[['first_name','elo', ' joined_date', "matches_played", "wins", "losses", "win_rate"]]
temp_df

,first_name,elo,joined_date,matches_played,wins,losses,win_rate
24,Robert Tamang,1281,17-09-8,41,35,6,85.37
19,Mons,1206,17-09-13,16,14,2,87.50
2,Andreas B.,1186,17-09-07,23,21,2,91.30
35,Harald Lønsethagen,1183,17-09-25,11,10,1,90.91
21,Ole Andreas,1137,17-09-11,25,17,8,68.00
9,Fabian S. Rønningen,1114,17-09-01,10,9,1,90.00
4,Bo Woelfert,1095,17-09-06,38,25,13,65.79
28,Thomas H,1090,17-09-3,9,6,3,66.67
37,Eirik,1075,17-09-28,5,4,1,80.00
30,Paul Frivold,1059,17-09-2,16,7,9,43.75


In [157]:
elo_df.columns

Index(['first_name', ' last_name', ' short_name', ' joined_date', ' id', 'elo',
       'matches_played', 'wins', 'losses', 'win_rate'],
      dtype='object')

In [25]:
latex_df = elo_df.sort_values('elo', ascending=False)[['first_name','elo',  ' joined_date',  "matches_played", "wins", "losses", "win_rate"]]
latex_df.columns = ["name", "elo", "joined_date", "matches_played", "wins", "losses", "win_rate"]
latex_df = latex_df.reset_index()
latex_df.index += 1
latex_df = latex_df[['name', 'elo', "joined_date", "matches_played", "wins", "losses", "win_rate"]]
latex_df_2 = latex_df.iloc[-5:]
static_index = latex_df.shape[0] - 4
elo_limit = latex_df.iloc[static_index-2]['elo'] # -2 since I set of index
print(elo_limit)
print(static_index)
latex_df_2 = latex_df_2.reset_index()
latex_df_2 = latex_df_2[['name', 'elo', "joined_date",  "matches_played", "wins", "losses", "win_rate"]]
latex_df_2['static_index'] = static_index
latex_df_2 = latex_df_2[['static_index', 'name', 'elo', "joined_date",  "matches_played", "wins", "losses", "win_rate"]]
latex_df_2['elo'] = "< " + str(elo_limit)
latex_df = latex_df.iloc[:-5]
latex_df.to

922
49


,name,elo,joined_date,matches_played,wins,losses,win_rate
1,Robert Tamang,1281,17-09-8,41,35,6,85.37
2,Mons,1206,17-09-13,16,14,2,87.50
3,Andreas B.,1186,17-09-07,23,21,2,91.30
4,Harald Lønsethagen,1183,17-09-25,11,10,1,90.91
5,Ole Andreas,1137,17-09-11,25,17,8,68.00
6,Fabian S. Rønningen,1114,17-09-01,10,9,1,90.00
7,Bo Woelfert,1095,17-09-06,38,25,13,65.79
8,Thomas H,1090,17-09-3,9,6,3,66.67
9,Eirik,1075,17-09-28,5,4,1,80.00
10,Paul Frivold,1059,17-09-2,16,7,9,43.75


In [27]:
latex_df.to_csv("elo_ordered_df.csv")

In [136]:
# make .tex file with table
latex_df.to_latex("latex_leaderboard/table_1.tex", index=True, columns=["name", "elo", "joined_date", "matches_played", "wins", "losses", "win_rate"],column_format="|r|l|r|l|l|l|l|l|", longtable=True)
latex_df_2.to_latex("latex_leaderboard/table_2.tex", index=False, columns=["static_index","name", "elo", "joined_date","matches_played", "wins", "losses", "win_rate"],column_format="|r|l|r|l|l|l|l|l|", longtable=True)


In [ ]:
temp_df

In [119]:
biggest_upset_loss_decrease

43.74876534991813

In [137]:
# funfacts

total_number_of_matches = matches_df.shape[0]
string = """

\begin{itemize}
    \item Total number of matches played: """ + str(total_number_of_matches) + """
    \item Biggest upset happened when """ + str(biggest_upset_player_win) + """ beat """ + str(biggest_upset_player_loss) + """. """ + str(biggest_upset_player_win) + """ gained """ + str(biggest_upset_win_increase) + """ elo, while """ + str(biggest_upset_player_loss) + """ lost """ + str(biggest_upset_loss_decrease) + """ elo. Date of match: """ + str(biggest_upset_player_date) + """
\end{itemize}
"""
print(string)



egin{itemize}
    \item Total number of matches played: 288
    \item Biggest upset happened when Odin A.S. beat Anders. Odin A.S. gained 43.75 elo, while Anders lost 43.75 elo . Date of match: 2017-10-02
\end{itemize}



# Save ELO-leaderboard to a csv file

# Notes

In [98]:
elo_df = elo_df.reset_index()

In [99]:
elo_df[elo_df['first_name'] == "Harald"]['elo']

Series([], Name: elo, dtype: int64)

In [56]:
elo_df = elo_df.set_index('first_name')
elo_df.set_value('Harald', 'elo', 3902)
elo_df.reset_index()

,first_name,index,last_name,short_name,joined_date,id,elo
0,Harald,0,Lønsethagen,hlø,17-09-05,1,3902
1,Mathias,1,Becksæther,mbe,17-09-05,2,1001


In [57]:
elo_df = elo_df.reset_index()

In [58]:
elo_df = elo_df.set_index('first_name')
elo_df.set_value('Harald', 'elo', 900)

,index,last_name,short_name,joined_date,id,elo
first_name,,,,,,
Harald,0,Lønsethagen,hlø,17-09-05,1,900
Mathias,1,Becksæther,mbe,17-09-05,2,1001


In [59]:
exp  = expected(1613, 1609)
exp += expected(1613, 1477)
exp

1.1920564660944573

In [23]:
elo(1613, exp, 2, k=32)  # 1601

1638.8541930849774

In [24]:
a = expected(1613, 1609)
temp_elo = elo(1613, a, 1, k=32)
print("temp_elo: " + str(temp_elo))
a = expected(temp_elo, 1477)
temp_elo = elo(temp_elo, a , 1, k=32)
print("temp_elo after second match: " + str(temp_elo))

temp_elo: 1628.8158013308434
temp_elo after second match: 1638.2378451931402


In [ ]:
# Defining new elo_table with 

# ===============
#          Notes 
# ===============

## Defining usefull DataFrames with Pandas
- All played matches

In [9]:
#matches_df = pd.dataFrame()
item_1 = pd.Series({'timestamp': '00-00-00',
                        'Player 1': 'Harald',
                        'Player 2': 'Petter',
                        'Winner': 'Harald'})
matches_df = pd.DataFrame([item_1])
matches_df.to_csv("matches.csv")
matches_df

,Player 1,Player 2,Winner,timestamp
0,Harald,Petter,Harald,00-00-00
